# Классификация изображений авто

Соревнование на Kaggle: **[[>](https://www.kaggle.com/c/sf-dl-car-classification)]**


# Imports and Settings

In [ ]:
!nvidia-smi

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import zipfile
import csv
import sys
import os
from datetime import timedelta, datetime as dt
from time import time

import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.regularizers import l2
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications import EfficientNetB0, EfficientNetB3
from tensorflow.keras.layers import *
from tensorflow.keras import Sequential as S
from sklearn.model_selection import train_test_split

# Увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 15, 10
# Улучшим вид графиков в svg
%config InlineBackend.figure_format = 'svg' 
%matplotlib inline


print(os.listdir("../input"))
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
print('Tensorflow   :', tf.__version__)
print('Keras        :', tf.keras.__version__)

In [ ]:
# Фиксируем версии всех пакетов для воспроизводимости
!pip freeze > requirements.txt

# Устаналиваем конкретное значение random seed для воспроизводимости
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)  
PYTHONHASHSEED = 0

DATA_PATH = '../input/sf-dl-car-classification/'
PATH = "../working/car/" # рабочая директория
os.makedirs(PATH, exist_ok=True)

sample_submission = pd.read_csv(DATA_PATH + 'sample-submission.csv')

train_df = pd.read_csv(DATA_PATH + 'train.csv')

In [ ]:
print('Распаковываем картинки.. ', end='')

for data_zip in ['train.zip', 'test.zip']:
    with zipfile.ZipFile(DATA_PATH + data_zip, "r") as z:
        z.extractall(PATH)
        
print('Готово')
print(os.listdir(PATH))

---
# EDA / Анализ данных

In [ ]:
# Для начала ознакомимся с данными:
print(train_df.head())
print()
print(train_df.info())

In [ ]:
train_df.Category

In [ ]:
train_df.Category.value_counts().plot(kind='barh', figsize=(3, 3));

Распределение классов достаточно равномерное - это хорошо

In [ ]:
print('Примеры картинок (random samples):')
plt.figure(figsize=(12, 8))

random_image = train_df.sample(n=9)
random_image_paths = random_image['Id'].values
random_image_cat = random_image['Category'].values

for index, path in enumerate(random_image_paths):
    im = PIL.Image.open(PATH + f'train/{random_image_cat[index]}/{path}')
    plt.subplot(3, 3, index + 1)
    plt.imshow(im)
    plt.title('Class: ' + str(random_image_cat[index]))
    plt.axis('off')
plt.show()

Посмотрим на пример картинки и её размер:

In [ ]:
image = PIL.Image.open(PATH + '/train/0/100380.jpg')
imgplot = plt.imshow(image)
plt.show()
image.size

---
# Подготовка данных

### Data Augmentation
Выполним аугментацию данных, что особенно полезно, если сэмплов не очень много.

In [ ]:
# Создание объектов генерации аугментированных изображений
def create_datagens():
    train_datagen = ImageDataGenerator(rotation_range=10,           # диапазон поворота в гр.
                                       brightness_range=[0.5, 1.5], # изменение яркости
                                       width_shift_range=0.1,       # диапазон сдвига в ширину
                                       height_shift_range=0.1,      # диапазон сдвига в высоту
                                       horizontal_flip=True,        # отражение по горизонтали
                                       validation_split=VAL_SPLIT)

    test_datagen = ImageDataGenerator()
    return train_datagen, test_datagen

### Data Generation

In [ ]:
# Обертка для генераторов данных
def rebuild_generators(): 
    # создаем объекты с аугментацией
    train_datagen, test_datagen = create_datagens()

    # генератор для тренировочной выборки
    train_generator = train_datagen.flow_from_directory(
        PATH + 'train/',    # директория, где расположены папки с картинками 
        target_size=(IMG_SIZE, IMG_SIZE), 
        batch_size=BATCH_SIZE, 
        class_mode='categorical', 
        shuffle=True, 
        seed=RANDOM_SEED, 
        subset='training'
    )

    # генератор для валидационной выборки
    validation_generator = train_datagen.flow_from_directory(
        PATH + 'train/', 
        target_size=(IMG_SIZE, IMG_SIZE), 
        batch_size=BATCH_SIZE, 
        class_mode='categorical', 
        shuffle=True, 
        seed=RANDOM_SEED, 
        subset='validation'
    ) 

    # генератор для тестовых данных
    test_subgenerator = test_datagen.flow_from_dataframe(
        dataframe=sample_submission, 
        directory=PATH + 'test_upload/', 
        x_col="Id", 
        y_col=None, 
        target_size=(IMG_SIZE, IMG_SIZE), 
        batch_size=BATCH_SIZE, 
        class_mode=None, 
        shuffle=False, 
        seed=RANDOM_SEED
    )
    return train_generator, validation_generator, test_subgenerator

---
# Model

### Callbacks Interface
https://keras.io/callbacks/

In [ ]:
# Созданим callback для фиксации длительности каждой эпохи
class TimingCallback(Callback):
    def __init__(self):
        self.logs=[]
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime=time()
    def on_epoch_end(self, epoch, logs={}):
        t = time() - self.starttime
        self.logs.append(round(t))

In [ ]:
# Определим функцию для создания списка callbacks
def recreate_callbacks():
    timing = TimingCallback()
    
    callbacks_list = [# сохранять прогресс обучения модели, чтобы 
                      # позже можно было подгрузить и дообучить модель:
                      ModelCheckpoint('best_model.hdf5',    
                                      monitor='val_accuracy', 
                                      verbose=1, 
                                      mode='max', 
                                      save_best_only=True), 
                      
                      # останавливать процесс обучения, если целевая 
                      # метрика не улучшается `patience` эпох подряд:
                      EarlyStopping(monitor='val_accuracy', 
                                    patience=4, 
                                    restore_best_weights=True), 
                      # снижать LR, если целевая метрика перестает улучаться
                      # (вместо него используется LearningRateScheduler)
#                       ReduceLROnPlateau(monitor='val_loss', 
#                                         factor=0.2, 
#                                         patience=3, 
#                                         min_lr=0.001),
                      
                      # постепенно уменьшать LR после каждой эпохи:
                      LearningRateScheduler(lambda x: LR * LR_DECAY_RATE ** x, 
                                            verbose=1), 
                      
                      # фиксировать тайминги эпох:
                      timing]
    return callbacks_list

### Helper Functions

#### Напишем несколько вспомогательных функций

#### Для визуализации и статистики:

In [ ]:
# Показывает отдельную метрику по эпохам
def show_metric(src):
    print({i + 1: round(src[i], 4) for i in range(len(src))})

# Показывает небольшую статистику прошедшего обучения
def show_stats(history):
    print('TRAINING STATS\n--------------\n')
    print(f'Base model: {base_model.name}; Optimizer: {model.optimizer._name}')
    print(f'IMG_SIZE: {IMG_SIZE}; BATCH_SIZE: {BATCH_SIZE};', 
          f'LR: {LR}; DROPOUT_RATE: {DROPOUT_RATE}')
    
    print('\nval_accuracy:')
    va = history.history['val_accuracy']
    show_metric(va)
    print('best:', round(max(va), 4))
    
    print('\nTimings:')
    show_metric(callbacks_list.timing.logs)
    
    print('\nTotal training time:')
    print(timedelta(seconds=sum(callbacks_list.timing.logs)))
    
# Отображение графиков прошедшего обучения
def plot_history(history):
    plt.style.use('Solarize_Light2')
    
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(acc))

    plt.figure(figsize=(6, 4))
    plt.plot(epochs, acc, 'b', label='Training acc')
    plt.plot(epochs, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()

    plt.figure(figsize=(6, 4))
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

    plt.show()

#### Для рутинных операций с моделью:

In [ ]:
# Сохранение результатов последнего обучения и подгрузка весов лучшей модели
# (также используется в fine-tuning для шаговых чекпоинтов)
def model_save(step=0):
    model.save(f'../working/model_step{step}.hdf5')
    model.load_weights('best_model.hdf5')
    
# Вспомогательная функция для оценки val_accuracy обученной модели
def model_evaluate():
    scores = model.evaluate_generator(test_generator, verbose=1)
    print('Accuracy: %.4f' % (scores[1]))

#### Для сборки и обучения модели:

In [ ]:
def model_assembly(base, head):
    '''
    Функция производит сборку модели по технике Transfer Learning.
    За основу берется предобученная сеть, на неё устанавливается новая 
    "голова" (head) из свежих слоев, которые понадобятся для решения текущей задачи.
    '''
    outputs = base.output
    
    for l in head.layers:
        outputs = l(outputs)

    return Model(inputs=base_model.input, 
                 outputs=outputs)

In [ ]:
def model_train(model, epochs):
    '''
    Шаблон для операции обучения модели (в т.ч. на шагах fine-tuning).
    Пересоздает список callbacks и запускает обучение модели.
    '''
    callbacks_list = recreate_callbacks()
    
    total_count = train_df.count()[0]
    val_count = int(total_count * VAL_SPLIT)
    train_count = total_count - val_count
    steps_per_epoch = train_count // BATCH_SIZE

    return model.fit(train_generator, 
                     steps_per_epoch=steps_per_epoch, 
                     validation_data=validation_generator, 
                     validation_steps=len(validation_generator), 
                     epochs=epochs, 
                     callbacks=callbacks_list)

### Hyperparameters
#### Определим ключевые гиперпараметры модели
*(в едином месте для удобства дальнейшего перебора)*

In [ ]:
IMG_SIZE             = 112    # размер подаваемого в сеть изображения
BATCH_SIZE           = 64     # размер Batch
VAL_SPLIT            = 0.15   # доля валидационной выборки

LR                   = 0.001  # Learning rate
LR_DECAY_RATE        = 0.9    # скорость "распада" Learning rate
DROPOUT_RATE         = 0.25   # размер Dropout
EPOCHS               = 15     # количество эпох на обучение


# Создадим генераторы данных на основе гиперпараметров
train_generator, validation_generator, test_subgenerator = rebuild_generators()

### Model Assembly

#### Загружаем предобученную на ImageNet сеть EfficientNetB0 
*(без "головы", т.к. будем ставить свою):*

In [ ]:
# base_model = InceptionResNetV2(weights='imagenet', 
base_model = EfficientNetB0(weights='imagenet', 
                            include_top=False, 
                            input_shape=(IMG_SIZE, IMG_SIZE, 3))

In [ ]:
# Задаем архитектуру "головы"
head = S([GlobalAveragePooling2D(), 
          
          Dense(128, use_bias=False, kernel_regularizer='l2'), 
          BatchNormalization(axis=1), 
          Activation('relu'), 

          Dropout(DROPOUT_RATE),
          Dense(10, activation='softmax')])


# Собираем модель
model = model_assembly(base_model, head)

# Компилируем
model.compile(loss='categorical_crossentropy', 
#               optimizer=optimizers.Adam(lr=LR), 
#               optimizer=optimizers.Adamax(lr=LR), 
#               optimizer=optimizers.Nadam(lr=LR), 
              optimizer=optimizers.Adam(lr=LR, amsgrad=True), 
              metrics=['accuracy'])

---
# Fit

#### Обучаем и экспериментируем:

*Следующий код представляет собой реализацию процесса экспериментального обучения модели.*

*Он закомментирован, чтобы не тратить ограниченные ресурсы GPU при запуске ноутбука на Kaggle.*

In [ ]:
history = model_train(model, epochs=EPOCHS)

In [ ]:
# show_stats(history)

In [ ]:
# сохраним итоговую сеть и подгрузим лучшую итерацию в обучении (best_model)
# model_save()
# model_evaluate()

#### Посмотрим графики обучения:

In [ ]:
# plot_history(history)

**С помощью экспериментов подобрали оптимальные параметры для болванки модели.**

**Теперь перейдем к Fine-tuning.**

---
# Fine-tuning

#### Применяем Transfer learning с Fine-tuning:

Сначала замораживаем все слои кроме новой "головы" и обучаем под новую задачу.

Затем будем последовательно размораживать сеть, обучая с уменьшенным Learning rate.

#### Завернем ключевые этапы Fine-tuning в функции для удобства в реализации шагов:

In [ ]:
def model_layers_info(model):
    '''
    Показывает количество слоев в модели, а также
    состояние открытости к обучению каждого слоя
    (✔️ - готов к обучению, ✖️ - заморожен)
    '''
    print(f'Model <{model._name}> layers count:', len(model.layers), 
          f'(trainable vars: {len(model.trainable_variables)})', end='\n\n')
    
    for i in range(len(model.layers)):
        l = model.layers[i]
        print(f'{i + 1:03}', f'✔️' if l.trainable else '✖️', l.name)

In [ ]:
def finetune(base_model, n, model, opt):
    '''
    Реализует шаг Fine-tuning, размораживая 'n' последних слоев у 
    'base_model' (BatchNorm-слои остаются нетронутыми).
    После этого модель 'model' компилируется в соответствии 
    с указанным оптимизатором 'opt'.
    '''
    # Отключаем обучаемость всей base_model
    base_model.trainable = False
    # Размораживаем n последних слоев
    for layer in base_model.layers[-n:]:
        # при fine-tuning BatchNorm-слои нужно оставлять замороженными
        if not isinstance(layer, BatchNormalization): 
            layer.trainable = True
    
    model.compile(loss="categorical_crossentropy", 
                  optimizer=opt, 
                  metrics=["accuracy"])

## Step 1

In [ ]:
IMG_SIZE   = 300
BATCH_SIZE = 32
LR         = 0.001
VAL_SPLIT  = 0.05


# Пересоздадим генераторы данных с новыми гиперпараметрами
train_generator, validation_generator, test_subgenerator = rebuild_generators()

#### Берем более эффективную SOTA-модель EfficientNetB3 в качестве базовой модели:

In [ ]:
# base_model = EfficientNetB7(weights='imagenet', 
base_model = EfficientNetB3(weights='imagenet', 
                            include_top=False, 
                            input_shape=(IMG_SIZE, IMG_SIZE, 3))

#### Сначала вручную собираем модель на базе полностью замороженной основы 

#### `base_model` (соответственно, веса, предобученные на imagenet, не затираются)

In [ ]:
# Для этого отключаем её обучаемость
base_model.trainable = False

# Задаем архитектуру "головы"
head = S([GlobalAveragePooling2D(), 
          
          Dense(128, use_bias=False, kernel_regularizer='l2'), 
          BatchNormalization(axis=1), 
          Activation('relu'), 

          Dropout(DROPOUT_RATE),
          Dense(10, activation='softmax')])

# Собираем модель
model = model_assembly(base_model, head)

# Компилируем
model.compile(loss="categorical_crossentropy", 
              optimizer=optimizers.Adam(lr=LR, amsgrad=True), 
              metrics=["accuracy"])

# Проверим результат
model_layers_info(model)

In [ ]:
# Обучаем
history = model_train(model, epochs=15)

In [ ]:
show_stats(history)

In [ ]:
model_save(step=1)
model_evaluate()

In [ ]:
plot_history(history)

## Step 2
#### На втором шаге разморозим половину слоев base_model и дообучим модель с уменьшенной Learning rate.

In [ ]:
LR = 0.0001


finetune(base_model, 
         n=int(len(base_model.layers) // 2),  # размораживаем половину слоев
         model=model, 
         opt=optimizers.Adam(lr=LR, amsgrad=True))

In [ ]:
# Посмотрим количество слоев и их статус обучаемости
model_layers_info(model)

In [ ]:
model.load_weights('best_model.hdf5')
model_evaluate()

In [ ]:
# Обучаем
history = model_train(model, epochs=10)

In [ ]:
show_stats(history)

In [ ]:
model_save(step=2)
model_evaluate()

In [ ]:
plot_history(history)

## Step 3
#### На третьем шаге разморозим все слои base_model и еще уменьшим Learning rate.

In [ ]:
LR = 0.00001

# Разморозим все слои base_model
finetune(base_model, 
         n=len(base_model.layers),  # размораживаем все слои
         model=model, 
         opt=optimizers.Adam(lr=LR, amsgrad=True))

In [ ]:
# Посмотрим количество слоев и их статус обучаемости
model_layers_info(model)

In [ ]:
model.load_weights('best_model.hdf5')
model_evaluate()

In [ ]:
# Обучаем
history = model_train(model, epochs=10)

In [ ]:
show_stats(history)

In [ ]:
model_save(step=3)
model_evaluate()

In [ ]:
plot_history(history)

## Step 4
#### На последнем шаге дообучаем сеть с увеличенным размером изображения.

Также придется уменьшить batch, иначе сеть не влезет в память GPU.

In [ ]:
IMG_SIZE             = 512
BATCH_SIZE           = 8
LR                   = 0.0001


# Пересоздаем генераторы данных с новыми гиперпараметрами
train_generator, validation_generator, test_subgenerator = rebuild_generators()

In [ ]:
# Загрузим предобученную сеть
base_model = EfficientNetB3(weights='imagenet', 
                            include_top=False, 
                            input_shape=(IMG_SIZE, IMG_SIZE, 3))

In [ ]:
# Задаем архитектуру "головы"
head = S([GlobalAveragePooling2D(), 
          
          Dense(128, use_bias=False, kernel_regularizer='l2'), 
          BatchNormalization(axis=1), 
          Activation("relu"), 
          Dropout(DROPOUT_RATE), 
          
          Dense(10, activation='softmax')])

# Собираем модель
model = model_assembly(base_model, head)

# Компилируем
model.compile(loss="categorical_crossentropy", 
              optimizer=optimizers.Adam(lr=LR, amsgrad=True), 
              metrics=["accuracy"])

In [ ]:
# Посмотрим количество слоев и их статус обучаемости
model_layers_info(model)

In [ ]:
model.load_weights('best_model.hdf5')
model_evaluate()

In [ ]:
# Обучаем
history = model_train(model, epochs=10)

In [ ]:
show_stats(history)

In [ ]:
model_save(step=4)
model_evaluate()

In [ ]:
plot_history(history)

# Предсказание на тестовых данных

In [ ]:
test_subgenerator.samples

In [ ]:
test_subgenerator.reset()

predictions = model.predict_generator(test_subgenerator, verbose=1) 
predictions = np.argmax(predictions, axis=-1)

label_map = (train_generator.class_indices)
label_map = dict((v, k) for k, v in label_map.items()) # flip k, v
predictions = [label_map[k] for k in predictions]

In [ ]:
submission = pd.DataFrame({'Id': test_subgenerator.filenames, 
                           'Category': predictions}, 
                          columns=['Id', 'Category'])

submission.head()

In [ ]:
# Сохраним submission
now = dt.now().strftime('[%d.%m.%Y]-[%H-%M]')
submission_name = f'submission-{now}.csv'
submission.to_csv(submission_name, index=False)

print('Submission saved')

# TTA (Test Time Augmentation)
https://towardsdatascience.com/test-time-augmentation-tta-and-how-to-perform-it-with-keras-4ac19b67fb4d

In [ ]:
model.load_weights('best_model.hdf5')

In [ ]:
test_datagen = ImageDataGenerator(rotation_range=10, 
                                  brightness_range=[0.5, 1.5], 
                                  width_shift_range=0.1, 
                                  height_shift_range=0.1, 
                                  horizontal_flip=True)

test_subgenerator = test_datagen.flow_from_dataframe(
    dataframe=sample_submission, 
    directory=PATH + 'test_upload/', 
    x_col='Id', 
    y_col=None, 
    target_size=(IMG_SIZE, IMG_SIZE), 
    batch_size=BATCH_SIZE, 
    class_mode=None, 
    shuffle=False, 
    seed=RANDOM_SEED
    )

In [ ]:
tta_steps = 10   # берем среднее за 10 предсказаний
predictions = []

for i in range(tta_steps):
    preds = model.predict_generator(test_subgenerator, verbose=1)
    predictions.append(preds)

pred = np.mean(predictions, axis=0)

In [ ]:
predictions = np.argmax(pred, axis=-1)

label_map = (train_generator.class_indices)
label_map = dict((v, k) for k, v in label_map.items()) # flip k, v

predictions = [label_map[k] for k in predictions]

submission = pd.DataFrame({'Id': test_subgenerator.filenames, 
                           'Category': predictions})

submission.head()

In [ ]:
# Сохраним submission
now = dt.now().strftime('[%d.%m.%Y]-[%H-%M]')
submission_name = f'submission-{now}-TTA.csv'
submission.to_csv(submission_name, index=False)

print('Submission saved')

In [ ]:
# Clean PATH
import shutil
shutil.rmtree(PATH)

# Заключение

- Основное тестирование для подбора гиперпараметров было решено проводить на сети EfficientNetB0

- Испытывались предобученные сети: EfficientNetE0, EfficientNetE3, EfficientNetE4, EfficientNetE6, EfficientNetE7;

Для финальной модели была выбрана EfficientNetE3 из-за оптимального рекомендованного размера входного изображения (300x300)

- Заметный прирост точности показало добавление в архитектуру "головы" BatchNormalization


- Пробовались разные способы Augmentation, оптимальные параметры были выстравлены для финальной модели


- Тестировались следующие оптимизаторы:
    - Adam
    - Nadam
    - Adamax

Лучший результат показал `Adam` с параметром `amsgrad=True`

- Добавлена l2-регуляризация


- Применены различные Keras CallBacks:
  - ModelCheckpoint
  - EarlyStopping
  - ReduceLROnPlateau
  - LearningRateScheduler
  - TimingCallback
  
    
- Испробованы разные техники управления LR:
  - callback: LearningRateScheduler()
  [[>]](https://keras.io/api/callbacks/learning_rate_scheduler/)
  - callback: ReduceLROnPlateau()
  [[>]](https://keras.io/api/callbacks/reduce_lr_on_plateau/)
  - tf.keras.optimizers.schedules.ExponentialDecay()
  [[>]](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/schedules/ExponentialDecay)

Для финального решения была выбрана настройка `LearningRateScheduler(lambda x: LR * LR_DECAY_RATE ** x, verbose=1)`


- Применена техника Test Time Augmentation, что дало небольшое улучшение точности.


Для настройки финальной модели было проведено большое количество экспериментов с разными значениями гиперпараметров (размер изображения, размер batch, Learning rate, количество эпох, оптимизаторы).

Fine-tuning показал себя хорошей работающей техникой, однако лучший результат в этом проекте был достигнут простым обучением в течение 20 эпох модели EfficientNetE3 с выставленными оптимальными для неё гиперпараметрами и с добавленной Test Time Augmentation.


---
Что не было сделано, но хотелось бы попробовать:

- Stratified-разбиение
- Динамическое увеличение размера картинки при Fine-tuning
- Другие стратегии разморозки слоев
- Cyclic Learning Rate
- Ансамблирование предобученных нейросетей
- Использование внешних датасетов для дообучения модели.